# Bullseye_二部探索

Google code jam 2013 round 1 Aの"bullseye"

問題の概要はここにある：https://codingcompetitions.withgoogle.com/codejam/round/0000000000432b32/0000000000432cd1


白と黒が同心円状に描かれた「矢の的のようなもの」をbullseyeという．

ここでは

- 一番内側は白で，黒い部分の輪の太さが1cmのbullseyeを描きたい．
- それぞれの黒い輪の間の間隔も1cmである．
- 1 mlの黒ペンキで$\pi \ \text{cm}^2$の面積を塗れる．
- 地は白である．

とする．

入力と，それに対する正しい出力の例は以下の通りである．

- 入力が$r = 1, \ t = 9$ならば，出力は$1$である．
- 入力が$r = 1, \ t = 10$ならば，出力は$2$である．
- 入力が$r = 3, \ t = 40$ならば，出力は$3$である．
- 入力が$r = 1, \ t = 1000000000000000000$ならば，出力は$707106780$である．
- 入力が$r = 10000000000000000, \ t = 1000000000000000000$ならば，出力は$49$である．


## 単純な解法: 2次方程式の解の公式の利用

まず単純に解いてみる．

* 高校までの数学の知識を使うと，簡単な計算により，一番内側の黒い輪を完成するためには$2 r + 1$のペンキが必要であるとわかる．
* 同様に内側から$k$本目の黒い輪を完成するためには$2 r + 4 k - 3$のペンキが必要であるとわかる．

よって$m$本の黒い輪から成るbullseyeを完成するには
$$
\sum_{k=1}^m (2 r + 4 k - 3) = 2 m^2 + (2 r - 1) m
$$
のペンキが必要であるとわかる．

よって$2 m^2 + (2 r - 1) m \le t$を満たす最大の$m$を出力すればよい．
二次方程式の解の公式から
$$
m = \left\lfloor \frac{1 - 2 r + \sqrt{(2 r - 1)^2 + 8 t}}{4}\right\rfloor
$$
を出力すればよい．

というのが素直な方針である．

この方針に従い問題を解く関数simple_answerを以下に定義する．

In [ ]:
import math

def simple_solution(r, t):
    m = (1 - 2 * r + math.sqrt((2 * r - 1) ** 2 + 8 * t)) // 4 # 上の式をそのままPythonで表しただけである．
    return int(m) # 組込み関数intの引数として小数を与えると，整数部分のみが返される．
    

In [ ]:
simple_solution(1, 9)

## 2分探索による解法の工夫

平方根の計算精度があてにならないので，平方根の計算は使わないことにする．

例えば「輪の本数$m$を仮定して$2 m^2 + (2 r - 1) m \le t$を満たすか否かを試す」という方針が考えられる．

輪の本数$m$は$0, 1, 2, \dots$と試しても良いが，正解の$m$が大きいときには計算時間がかかるのではないかと心配になる．

よってここでは$m$に関する2分探索を試すことにする．

以下に，まず与えられた$m, r, t$が$2 m^2 + (2 r - 1) m \le t$を満たすか否かを判定する関数is_possibleを定義し，次にそれを用いて解答を2分探索する関数solution_by_binary_searchを定義する．

In [ ]:
'''輪の本数mと中心の白い部分の半径rとペンキの量tが与えられたとき、輪をm本塗れるならばTrue、そうでないならばFalseを返す関数'''

def is_possilbe(m, r, t):
  if 2*(m**2)+(2*r-1)*m <= t:
    return True
  else:
    return False

In [ ]:
'''2分探索でギリギリのm、すなわち解答を返す関数. 引数のlower_boundにはmの下界、upper_boundにはmの上界を与える'''
def solution_by_binary_search(lower_bound, upper_bound, r, t):
  # mの上界と下界が離れている間は，mの候補を絞るため以下を繰り返す
  while upper_bound - lower_bound > 1:
    # 上界と下界の中間（の小数部分切り捨て）を計算する
    middle = (upper_bound + lower_bound) // 2
    
    # 中間が「塗れる」本数ならば,ギリギリのmは中間と上界の間にあるはずなので,新たに中間の本数を下界とする
    if is_possible(middle, r, t) == True:
      lower_bound = middle
    # そうでないならば,ギリギリのmは下界と中間の間にあるはずなので，新たに中間の本数を上界とする．
    else:
      upper_bound = middle
  '''
    繰り返しが終わったならば、mの候補は上界か下界のいずれかになっているはず
  '''
  # 上界が「塗れる」本数ならば上界を解答として返す
  if is_possible(upper_bound, r, t) == True:
    return upper_bound
  # 下界が「塗れる」本数ならば下界を解答として返す
  else:
    return lower_bound

In [ ]:
# 関数solution_by_binary_searchを実行する際には，自明な下界として0，自明な上界としてtを使う．
solution_by_binary_search(0, 9, 1, 9)

1

この関数solution_by_binary_searchを用いて，入力データから出力を得るコードを以下に記す．

ここでは，Google code jamのオンラインジャッジを使うことを想定し，標準入出力対応のスクリプトコードにした

In [ ]:
T = int(input())
for case_number in range(T):
    r, t = map(int, input().split())
    print(f'Case #{case_number + 1}: {solution_by_binary_search(0, t, r, t)}')

5
1 9
Case #1: 1
1 9
Case #2: 1
3 40
Case #3: 3
1 1000000000000000000
Case #4: 707106780
10000000000000000 1000000000000000000
Case #5: 49
